In [1]:
#ACTUAL TASK
import time
import cv2
import mediapipe as mp
import numpy as np

#to calculate angle between the landmarks passed to this function
def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
     
    if angle >180.0:
        angle=360-angle
        
    return angle



mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

counter=0
flag=0
flag1=0
stage=None

start=time.time()

frameSize = (640,854)
cap=cv2.VideoCapture("KneeBendVideo.mp4")
cap.set(cv2.CAP_PROP_FPS, 25)
out = cv2.VideoWriter('output3.mp4',cv2.VideoWriter_fourcc('M','J','P','G'),25 , (854,640))

with mp_pose.Pose(min_detection_confidence=0.7,min_tracking_confidence=0.7)as pose:
    while cap.isOpened():
        ret,frame=cap.read() #capturing frames from the video.
        
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False
        
        #make detections
        results=pose.process(image) #to obtain 33 landmarks present on body in the image
        
        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        
        try:
            landmarks=results.pose_landmarks.landmark
            #To obtain x&y co-ordinates of landmarks specified.
            hip=[landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee=[landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle=[landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
           
            angle=calculate_angle(hip,knee,ankle)#passing 3 landmarks(hip,knee & ankle) to get angle at knee from calculate_angle() function
           
            cv2.putText(image,str(angle),
                       tuple(np.multiply(knee,[640,540]).astype(int)),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
                        )
            
            #To calculate the number of REPS based on the condition for threshold angle(i.e 140 degrees)
            if angle>140:
                stage="down"

                flag=1
                flag1=0
            if angle<140 and stage=='down':
                stage='up'
                if(flag==1):
                    start=time.time()#To start measuring time when knee is bent
                    flag1=1
                flag=0
                counter+=1
                
            
            
        except:
            pass
        
        stop=time.time()
        
        total_time=stop-start
        if(flag1==0 and flag==1 ):
            if(int(total_time)<8):#to check condition when knee is not bent.
                        cv2.rectangle(image,(323,0),(520,33),(20,166,166),-1)
                        cv2.putText(image,'Keep your knees bent',(330,15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0) , 1 ,cv2.LINE_AA)
                        
                        
        
        cv2.rectangle(image,(0,0),(160,65),(245,117,16),-1)
        
        cv2.putText(image,'REPS',(15,12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0) , 1 ,cv2.LINE_AA)
        cv2.putText(image,str(counter),(10,50),
                   cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,0),2,cv2.LINE_AA)
        
        cv2.putText(image,'STAGE',(85,12),
           cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0) , 1 ,cv2.LINE_AA)
        cv2.putText(image,stage,(80,40),
                   cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2,cv2.LINE_AA)
        
        cv2.rectangle(image,(161,0),(322,65),(20,17,166),-1)
        cv2.putText(image,'Time',(210,18),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.75 , (0,0,0) , 1 ,cv2.LINE_AA)
        cv2.putText(image,str(int(total_time))+'s',(205,60),
                   cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,0),2,cv2.LINE_AA)
        
         

        
        
        #Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        
        
        cv2.imshow('Mediapipe Feed',image)

        out.write(image)

        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
